### 1) Import Data

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
df_train = pd.read_pickle('../train.pkl')
df_train.shape

(1460, 328)

### 1.1) Data Transformation

In [3]:
df_test = pd.read_pickle('../test.pkl')
df_test.shape

(1459, 327)

### 2) Setting up Data for Training

In [4]:
X = df_train.drop('SalePrice', axis = 1)
y = df_train['SalePrice']

In [5]:
y.shape

(1460,)

In [6]:
X.shape

(1460, 327)

### 3) Create Random Noise Data

In [7]:
X_1 = df_train.drop('SalePrice', axis=1)
X_1['Random_Noise'] = np.random.random(size=len(X_1))
y_1 = df_train['SalePrice']

In [8]:
y_1.shape

(1460,)

In [9]:
X_1.shape

(1460, 328)

In [22]:
Z_1 = df_test
Z_1['Random_Noise'] = np.random.random(size=len(Z_1))

### 4) Scaling and Running Elastic Nest

### 4.1) Running Scaling and Data Transformation

In [10]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
y = np.log(y)

In [11]:
scale = StandardScaler()
scale.fit(X)
X1 = scale.transform(X)

In [12]:
scale.fit(df_test)
z1 = scale.transform(df_test)

In [13]:
y_1 = np.log(y_1)

In [14]:
scale = StandardScaler()
scale.fit(X_1)
X_2 = scale.transform(X_1)

In [23]:
scale.fit(Z_1)
Z_2 = scale.transform(Z_1)

### 4.2) Running Elastic Nest

In [17]:
from sklearn.linear_model import ElasticNet
regr = ElasticNet(random_state=0)
regr.fit(X1, y)
ElasticNet(alpha=1.0, fit_intercept=True, l1_ratio=1.0,
      max_iter=1000000, random_state=42, selection='random')
regr.score(X1,y)

0.0

In [16]:
from sklearn.linear_model import ElasticNetCV
#alpha = 1
elasregrCV= ElasticNetCV(alphas = [0.0001,0.001,0.01,0.1,1], 
                         l1_ratio= [.1, .5, .7, .9, .95, .99, .995, 1], 
                         max_iter=100000, fit_intercept = True, selection = 'random',
                         random_state=42)
elasregrCV.fit(X1, y)
print(elasregrCV.score(X1, y))
#Since taking the log, so we need to scale back using exponential.
np.exp(elasregrCV.predict(z1))

0.9463046673654247


array([134926.10395489, 161517.45092244, 192164.07871572, ...,
       165989.13064458, 132509.19870633, 203876.87193353])

In [71]:
len(np.exp(elasregrCV.predict(z1)))

1459

In [72]:
elasregrCV1= ElasticNetCV(alphas = [0.0001,0.001,0.01,0.1,1], 
                         l1_ratio= [.1, .5, .7, .9, .95, .99, .995, 1], 
                         max_iter=100000, fit_intercept = True, selection = 'random',
                         random_state=42)
elasregrCV1.fit(X_2, y_1)
print(elasregrCV1.score(X_2, y_1))
len(np.exp(elasregrCV1.predict(Z_2)))

0.9463193080943417


1459

In [ ]:
results = pd.DataFrame(np.exp(elasregrCV.predict(z1)))
results.head(5)

In [80]:
results2 = pd.DataFrame(np.exp(elasregrCV1.predict(Z_2)))
results2.to_csv("kagglesample.csv")

In [61]:
l = pd.DataFrame([i for i in range(1461,2919)])
k = l.reset_index()
k.head()

,index,0
0,0,1461
1,1,1462
2,2,1463
3,3,1464
4,4,1465


In [53]:
results2.head()

,index,0
0,0,135080.363376
1,1,161616.654602
2,2,191952.225672
3,3,191470.297291
4,4,188243.452638


In [65]:
totalresult = pd.merge(k,results2, on = 'index')
totalresult.head()
del(totalresult['index'])
totalresult.head()

,0_x,0_y
0,1461,135080.363376
1,1462,161616.654602
2,1463,191952.225672
3,1464,191470.297291
4,1465,188243.452638


In [67]:
totalresult2 = totalresult.rename(index=str, columns={"0_x": "ID", "0_y": "SalePrice"})

In [69]:
totalresult2.to_csv("kaggle1.csv")

In [81]:
%notebook "C:\\Users\\Bird_\\Desktop\\Data Science Bootcamp\\Project3\\kaggle-housing-prices\\Bird\\ElasticNet.ipynb" 